In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import string
import re

import statsmodels.formula.api as smf
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import ElasticNet
from sklearn.model_selection import KFold
from sklearn.metrics import mean_squared_error, mean_absolute_error
from sklearn.metrics import mean_absolute_percentage_error

# Limpieza

In [ ]:
#Subimos nuestras base de comments filtrada por positivos y negativos
reviews = 'https://media.githubusercontent.com/media/AdrianaRODA/4Geeks_NLP_Airbnb_Reviews/refs/heads/main/Data/base_comments_filtrada2.csv'
amenities = 'https://media.githubusercontent.com/media/AdrianaRODA/4Geeks_NLP_Airbnb_Reviews/refs/heads/main/Data/Amenities.csv'

In [ ]:
df_reviews = pd.read_csv(reviews)
df_amenities = pd.read_csv(amenities)

In [ ]:
df_amenities.columns

Index(['Unnamed: 0', 'id', 'room_type', 'beds', 'bedrooms', 'bathrooms',
       'price', 'amenities', 'AMENITIES', 'categorias_estandar', 'wifi', 'tv',
       'cocina basica', 'refrigerador', 'cocina extra', 'basicos baño',
       'belleza', 'servicio limpieza', 'estacionamiento', 'centro lavado',
       'sound system', 'bbq grill', 'basicos cuarto', 'pool', 'consola',
       'streaming', 'chimenea', 'espacio de trabajo', 'cortinas',
       'seguridad externa', 'seguridad interna', 'ambiente', 'baño equipado',
       'ejercicio', 'exterior', 'extras', 'niños', 'room_type_Entire home/apt',
       'room_type_Hotel room', 'room_type_Private room',
       'room_type_Shared room'],
      dtype='object')

In [ ]:
df_amenities.rename(columns={'id':'listing_id'}, inplace=True)

In [ ]:
df_amenities.columns

Index(['Unnamed: 0', 'listing_id', 'room_type', 'beds', 'bedrooms',
       'bathrooms', 'price', 'amenities', 'AMENITIES', 'categorias_estandar',
       'wifi', 'tv', 'cocina basica', 'refrigerador', 'cocina extra',
       'basicos baño', 'belleza', 'servicio limpieza', 'estacionamiento',
       'centro lavado', 'sound system', 'bbq grill', 'basicos cuarto', 'pool',
       'consola', 'streaming', 'chimenea', 'espacio de trabajo', 'cortinas',
       'seguridad externa', 'seguridad interna', 'ambiente', 'baño equipado',
       'ejercicio', 'exterior', 'extras', 'niños', 'room_type_Entire home/apt',
       'room_type_Hotel room', 'room_type_Private room',
       'room_type_Shared room'],
      dtype='object')

In [ ]:
sentimientos_por_listing = df_reviews.groupby('listing_id')['porcentaje'].mean()

In [ ]:
sentimientos_por_listing.nunique()

4798

In [ ]:
pd.DataFrame(sentimientos_por_listing)

,porcentaje
listing_id,
165772,91.666667
171109,97.619048
180808,100.000000
245507,96.750000
246057,98.823529
...,...
1376211426481638186,100.000000
1376318084540799222,100.000000
1376450215192500292,100.000000


In [ ]:
alcaldia = df_reviews[['listing_id','neighbourhood_cleansed']]
print(alcaldia.nunique())
alcaldia.shape

listing_id                16312
neighbourhood_cleansed       16
dtype: int64


(253876, 2)

In [ ]:
#inner es el que solo agarra los coincidentes y ese va en el "how" (how='inner')
df_merge = pd.merge(sentimientos_por_listing, df_amenities, on='listing_id', how='inner')

In [ ]:
total_reviews = df_reviews.groupby('listing_id')['comments'].count()

In [ ]:
df_merge_reviews = pd.merge(df_merge, total_reviews, on='listing_id', how='inner')

In [ ]:
df_merge_reviews = pd.merge(df_merge_reviews, alcaldia, on='listing_id', how='inner')

In [ ]:
df_merge_reviews

,listing_id,porcentaje,Unnamed: 0,room_type,beds,bedrooms,bathrooms,price,amenities,AMENITIES,...,ejercicio,exterior,extras,niños,room_type_Entire home/apt,room_type_Hotel room,room_type_Private room,room_type_Shared room,comments,neighbourhood_cleansed
0,165772,91.666667,6,Entire home/apt,10.0,5.0,5.0,"$3,982.00","[""Children\u2019s books and toys"", ""Pocket wif...","childrenus books and toys,pocket wifi,hair dry...",...,0,1,1,1,1.0,0.0,0.0,0.0,3,Miguel Hidalgo
1,165772,91.666667,6,Entire home/apt,10.0,5.0,5.0,"$3,982.00","[""Children\u2019s books and toys"", ""Pocket wif...","childrenus books and toys,pocket wifi,hair dry...",...,0,1,1,1,1.0,0.0,0.0,0.0,3,Miguel Hidalgo
2,165772,91.666667,6,Entire home/apt,10.0,5.0,5.0,"$3,982.00","[""Children\u2019s books and toys"", ""Pocket wif...","childrenus books and toys,pocket wifi,hair dry...",...,0,1,1,1,1.0,0.0,0.0,0.0,3,Miguel Hidalgo
3,171109,97.619048,7,Private room,1.0,1.0,1.0,$327.00,"[""Hair dryer"", ""Hangers"", ""Microwave"", ""Cookin...","hair dryer,hangers,microwave,cooking basics,pa...",...,0,1,1,0,0.0,0.0,1.0,0.0,3,Benito Juárez
4,171109,97.619048,7,Private room,1.0,1.0,1.0,$327.00,"[""Hair dryer"", ""Hangers"", ""Microwave"", ""Cookin...","hair dryer,hangers,microwave,cooking basics,pa...",...,0,1,1,0,0.0,0.0,1.0,0.0,3,Benito Juárez
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
253653,1376211426481638186,100.000000,25938,Entire home/apt,2.0,2.0,2.0,"$1,257.00","[""Wine glasses"", ""Hangers"", ""Cooking basics"", ...","wine glasses,hangers,cooking basics,microwave,...",...,1,1,1,1,1.0,0.0,0.0,0.0,1,Álvaro Obregón
253654,1376318084540799222,100.000000,25943,Private room,1.0,1.0,1.0,$951.00,"[""Wine glasses"", ""Hangers"", ""Pets allowed"", ""M...","wine glasses,hangers,pets allowed,microwave,mi...",...,1,0,1,0,0.0,0.0,1.0,0.0,1,Cuauhtémoc
253655,1376450215192500292,100.000000,25963,Entire home/apt,3.0,2.0,2.0,"$2,800.00","[""Wine glasses"", ""Hair dryer"", ""Hangers"", ""Mic...","wine glasses,hair dryer,hangers,microwave,cook...",...,0,0,1,0,1.0,0.0,0.0,0.0,1,Miguel Hidalgo
253656,1376960821808863954,100.000000,25998,Private room,2.0,1.0,1.0,"$1,246.00","[""Wine glasses"", ""Hangers"", ""Pets allowed"", ""M...","wine glasses,hangers,pets allowed,microwave,mi...",...,1,0,1,0,0.0,0.0,1.0,0.0,1,Cuauhtémoc


# Regresion

In [ ]:
url = 'https://media.githubusercontent.com/media/AdrianaRODA/4Geeks_NLP_Airbnb_Reviews/refs/heads/main/Data/base_price_deviation.csv'

In [ ]:
df_price = pd.read_csv(url)

In [ ]:
def to_binary(X_train_, X_test_, cats):
  X_train =  X_train_.copy()
  X_test = X_test_.copy()
  ohe = OneHotEncoder(handle_unknown='ignore', drop='first')
  # train
  X_train_bin = pd.DataFrame(ohe.fit_transform(
      X_train[cats]).toarray(),
      columns = ohe.get_feature_names_out(),
      index = X_train.index)
  X_train_bin = X_train_bin.join(X_train[list(set(X_train.columns)  - set(cats))])
  # test
  X_test_bin = pd.DataFrame(ohe.transform(X_test[cats]).toarray(),
      columns = ohe.get_feature_names_out(),
      index = X_test.index)
  X_test_bin = X_test_bin.join(X_test[list(set(X_test.columns)  - set(cats))])
  X_test_bin = X_test_bin[X_train_bin.columns]
  return X_train_bin, X_test_bin

def Elastic_gridcv(X_train, y_train):
    model =  ElasticNet(random_state=321)
    hyperparams = {"alpha" :  [0.01, 100, 250],
                   "l1_ratio" :  np.linspace(0,1,10),
                   "max_iter": [200, 250, 300],
                   "selection": ['cyclic', 'random'],
                   "tol": [ 1e-10],}
    cv = KFold(n_splits=5, shuffle=True, random_state=123) # replicables...
    grid_search = GridSearchCV(estimator=model,
                               param_grid=hyperparams,
                               cv=cv,
                               scoring= 'neg_mean_squared_error',)
    grid_result = grid_search.fit(X_train, y_train)
    return grid_search.best_estimator_




In [ ]:
df_price['price'] = df_price['price'].astype(str).str.replace('[$,]', '',regex=True)
df_price['price'] = pd.to_numeric(df_price['price'])

y = df_price['price']

In [ ]:
ohe = OneHotEncoder()
df_encoded = pd.DataFrame(ohe.fit_transform(df_price[["neighbourhood_cleansed"]]).toarray(),
                          columns=ohe.get_feature_names_out(['neighbourhood_cleansed']),
                          index =df_price.index )

df_price = pd.concat([df_price, pd.DataFrame(df_encoded)], axis=1)
df_price

,listing_id,porcentaje,Unnamed: 0,room_type,beds,bedrooms,bathrooms,price,amenities,AMENITIES,...,neighbourhood_cleansed_Iztacalco,neighbourhood_cleansed_Iztapalapa,neighbourhood_cleansed_La Magdalena Contreras,neighbourhood_cleansed_Miguel Hidalgo,neighbourhood_cleansed_Milpa Alta,neighbourhood_cleansed_Tlalpan,neighbourhood_cleansed_Tláhuac,neighbourhood_cleansed_Venustiano Carranza,neighbourhood_cleansed_Xochimilco,neighbourhood_cleansed_Álvaro Obregón
0,165772,91.666667,6,Entire home/apt,10.0,5.0,5.0,3982.0,"[""Children\u2019s books and toys"", ""Pocket wif...","childrenus books and toys,pocket wifi,hair dry...",...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
1,165772,91.666667,6,Entire home/apt,10.0,5.0,5.0,3982.0,"[""Children\u2019s books and toys"", ""Pocket wif...","childrenus books and toys,pocket wifi,hair dry...",...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
2,165772,91.666667,6,Entire home/apt,10.0,5.0,5.0,3982.0,"[""Children\u2019s books and toys"", ""Pocket wif...","childrenus books and toys,pocket wifi,hair dry...",...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
3,171109,97.619048,7,Private room,1.0,1.0,1.0,327.0,"[""Hair dryer"", ""Hangers"", ""Microwave"", ""Cookin...","hair dryer,hangers,microwave,cooking basics,pa...",...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,171109,97.619048,7,Private room,1.0,1.0,1.0,327.0,"[""Hair dryer"", ""Hangers"", ""Microwave"", ""Cookin...","hair dryer,hangers,microwave,cooking basics,pa...",...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
253653,1376211426481638186,100.000000,25938,Entire home/apt,2.0,2.0,2.0,1257.0,"[""Wine glasses"", ""Hangers"", ""Cooking basics"", ...","wine glasses,hangers,cooking basics,microwave,...",...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
253654,1376318084540799222,100.000000,25943,Private room,1.0,1.0,1.0,951.0,"[""Wine glasses"", ""Hangers"", ""Pets allowed"", ""M...","wine glasses,hangers,pets allowed,microwave,mi...",...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
253655,1376450215192500292,100.000000,25963,Entire home/apt,3.0,2.0,2.0,2800.0,"[""Wine glasses"", ""Hair dryer"", ""Hangers"", ""Mic...","wine glasses,hair dryer,hangers,microwave,cook...",...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
253656,1376960821808863954,100.000000,25998,Private room,2.0,1.0,1.0,1246.0,"[""Wine glasses"", ""Hangers"", ""Pets allowed"", ""M...","wine glasses,hangers,pets allowed,microwave,mi...",...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [ ]:
df_price.columns

Index(['listing_id', 'porcentaje', 'Unnamed: 0', 'room_type', 'beds',
       'bedrooms', 'bathrooms', 'price', 'amenities', 'AMENITIES',
       'categorias_estandar', 'wifi', 'tv', 'cocina basica', 'refrigerador',
       'cocina extra', 'basicos baño', 'belleza', 'servicio limpieza',
       'estacionamiento', 'centro lavado', 'sound system', 'bbq grill',
       'basicos cuarto', 'pool', 'consola', 'streaming', 'chimenea',
       'espacio de trabajo', 'cortinas', 'seguridad externa',
       'seguridad interna', 'ambiente', 'baño equipado', 'ejercicio',
       'exterior', 'extras', 'niños', 'room_type_Entire home/apt',
       'room_type_Hotel room', 'room_type_Private room',
       'room_type_Shared room', 'comments', 'neighbourhood_cleansed',
       'neighbourhood_cleansed_Azcapotzalco',
       'neighbourhood_cleansed_Benito Juárez',
       'neighbourhood_cleansed_Coyoacán',
       'neighbourhood_cleansed_Cuajimalpa de Morelos',
       'neighbourhood_cleansed_Cuauhtémoc',
       'neighb

In [ ]:
df_price['neighbourhood_cleansed_Cuauhtémoc']

,neighbourhood_cleansed_Cuauhtémoc
0,0.0
1,0.0
2,0.0
3,0.0
4,0.0
...,...
253653,0.0
253654,1.0
253655,0.0
253656,1.0


In [ ]:
print(df_price['price'].min())
print(df_price['price'].max())

149.0
438053.0


In [ ]:
# Variables independientes
X = df_price.drop(columns=['listing_id','neighbourhood_cleansed','price','Unnamed: 0', 'room_type', 'amenities', 'AMENITIES', 'categorias_estandar', 'room_type_Private room', 'neighbourhood_cleansed_Álvaro Obregón'])

In [ ]:
X.columns

Index(['porcentaje', 'beds', 'bedrooms', 'bathrooms', 'wifi', 'tv',
       'cocina basica', 'refrigerador', 'cocina extra', 'basicos baño',
       'belleza', 'servicio limpieza', 'estacionamiento', 'centro lavado',
       'sound system', 'bbq grill', 'basicos cuarto', 'pool', 'consola',
       'streaming', 'chimenea', 'espacio de trabajo', 'cortinas',
       'seguridad externa', 'seguridad interna', 'ambiente', 'baño equipado',
       'ejercicio', 'exterior', 'extras', 'niños', 'room_type_Entire home/apt',
       'room_type_Hotel room', 'room_type_Shared room', 'comments',
       'neighbourhood_cleansed_Azcapotzalco',
       'neighbourhood_cleansed_Benito Juárez',
       'neighbourhood_cleansed_Coyoacán',
       'neighbourhood_cleansed_Cuajimalpa de Morelos',
       'neighbourhood_cleansed_Cuauhtémoc',
       'neighbourhood_cleansed_Gustavo A. Madero',
       'neighbourhood_cleansed_Iztacalco', 'neighbourhood_cleansed_Iztapalapa',
       'neighbourhood_cleansed_La Magdalena Contreras',

In [ ]:
# Dividir los datos en conjuntos de entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [ ]:
# X_train_bin, X_test_bin = to_binary(X_train, X_test, ['neighbourhood_cleansed'])

# X_train_bin

In [ ]:
#Checar el tema de la regresion y como obtener el precio predicho para obtener el price deviation
# Crear y entrenar el modelo de regresión lineal
modelo_hedonico = LinearRegression()
modelo_hedonico.fit(X_train, y_train)

# Predecir los precios para todo el DataFrame
df_price['precio_predicho'] = modelo_hedonico.predict(X)

print("\nDataFrame con precios predichos:")
print(df_price[['price', 'precio_predicho']].head())


DataFrame con precios predichos:
    price  precio_predicho
0  3982.0      4773.493917
1  3982.0      4773.493917
2  3982.0      4773.493917
3   327.0       356.424985
4   327.0       356.424985


In [ ]:
#modelo = Elastic_gridcv(X_train_bin, y_train)

In [ ]:
y_pred = modelo_hedonico.predict(X_test_bin)

NameError: name 'X_test_bin' is not defined

In [ ]:
mean_absolute_error(y_test, y_pred)

In [ ]:
mean_absolute_percentage_error(y_test, y_pred)

In [ ]:
# Calcular el Pricing Deviation
df_price['pricing_deviation'] = df_price['price'] - df_price['precio_predicho']

print("\nDataFrame con el Pricing Deviation:")
print(df_price[['price', 'precio_predicho', 'pricing_deviation']].head())

In [ ]:
price_alcaldia = df_price.groupby('neighbourhood_cleansed').agg(
    mean_pricing_deviation=('pricing_deviation', 'mean'),
    comments_count=('comments', 'sum'))

pd.DataFrame(price_alcaldia)

In [ ]:
correlacion = price_alcaldia['mean_pricing_deviation'].corr(price_alcaldia['comments_count'])
print(f"\nCorrelación entre Pricing Deviation y Número de Reviews: {correlacion:.2f}")

# Visualizar la correlación
plt.figure()
sns.scatterplot(x='comments_count', y='mean_pricing_deviation', data=price_alcaldia, alpha=0.6)
plt.title('Correlación entre Desviación de Precios y Volumen de Reviews')
plt.xlabel('Número de Reviews')
plt.ylabel('Desviación de Precios (Pricing Deviation)')
plt.axhline(0, color='red', linestyle='--')
plt.grid(True)
plt.show()